In [1]:
%%html
<style>
div.output_area pre {
    white-space: pre;
    word-wrap: normal;
}
.container { 
    width:100% !important; 
}
</style>

In [2]:
import datetime
import numpy as np
import glob, json
from numpy import nan
import pandas as pd

In [3]:
Matrix = []
Matrix.append([3,5,7,7])
Matrix.append([5,5,7,7])
Matrix.append([7,7,7,7])
Matrix.append([7,7,7,7])

In [4]:
list_index = {0:'Metro Manila',1:'Luzon',2:'Visayas',3:'Mindanao'}
list_index_city = {'Metro Manila':0,'Luzon':1,'Visayas':2,'Mindanao':3}

In [34]:
def get_city(address):
    index = []
    for key in list_index:
        index.append(address.lower().rfind(list_index[key].lower()))
    maxIndex = -1
    result = ""
    for i in range(0,4):
        if index[i] > maxIndex:
            maxIndex = index[i]
            result = list_index[i]
    return result
#udf_get_city = udf(get_city)

In [33]:
def distance(city_seller_address,buyeraddress):
    return Matrix[list_index_city[city_seller_address]][list_index_city[buyeraddress]]
#udf_estimate = udf(distance,IntegerType())

In [24]:
list_dates = {}
i = 0
startDate = datetime.date(2020, 3, 1)
while startDate.month < 5:
    list_dates[startDate.strftime('%Y-%m-%d')]=i+1
    startDate += datetime.timedelta(days=1)
    i+=1

In [25]:
list_offday = ['2020-03-01','2020-03-08','2020-03-15','2020-03-22','2020-03-25','2020-03-29','2020-03-30','2020-03-31','2020-04-05','2020-04-12','2020-04-19','2020-04-26']

In [26]:
def diffTime(startDate,endDate):
    begin = list_dates[startDate]
    end = list_dates[endDate]
    offDay = 0
    for offday in list_offday:
        if list_dates[offday] > begin and list_dates[offday] <= end:
            offDay +=1
    return (end -begin - offDay)

In [27]:
def isLate(day_pick,day_1st_deliver_attempt_timestamp,day_2nd_deliver_attempt_timestamp,SLA):
    if diffTime(day_pick,day_1st_deliver_attempt_timestamp) > SLA:
        return 1
    if diffTime(day_pick,day_1st_deliver_attempt_timestamp) <= SLA and day_2nd_deliver_attempt_timestamp !='NaT' and diffTime(day_1st_deliver_attempt_timestamp,day_2nd_deliver_attempt_timestamp) >3:
        return 1
    return 0

In [29]:
df_listorder = pd.read_csv('./data/delivery_orders_march.csv')

In [30]:
df_listorder.columns

Index(['orderid', 'pick', '1st_deliver_attempt', '2nd_deliver_attempt',
       'buyeraddress', 'selleraddress'],
      dtype='object')

In [31]:
df_listorder['city_buyer_address']= df_listorder['buyeraddress'].apply(lambda x: get_city(x))

In [32]:
df_listorder['city_seller_address']= df_listorder['selleraddress'].apply(lambda x: get_city(x))

In [35]:
df_listorder['SLA'] = df_listorder.apply(lambda x: distance(x['city_seller_address'], x['city_buyer_address']), axis=1)

In [36]:
df_listorder['pick_timestamp8']= df_listorder['pick'].apply(lambda x: x+ 3600 if x is not None else None)

In [37]:
df_listorder['day_pick']= pd.to_datetime(df_listorder['pick_timestamp8'], unit='s').apply(lambda x: str(x)[:10] if x is not None else None)

In [38]:
df_listorder['1st_deliver_attempt_timestamp8']= df_listorder['1st_deliver_attempt'].apply(lambda x: x+ 3600 if x is not None else None)

In [39]:
df_listorder['day_1st_deliver_attempt']= pd.to_datetime(df_listorder['1st_deliver_attempt_timestamp8'], unit='s').apply(lambda x: str(x)[:10]if x is not None else None)

In [40]:
df_listorder['2nd_deliver_attempt_timestamp8']= df_listorder['2nd_deliver_attempt'].apply(lambda x: x+ 3600 if x is not None else None)

In [41]:
df_listorder['day_2nd_deliver_attempt']= pd.to_datetime(df_listorder['2nd_deliver_attempt_timestamp8'], unit='s').apply(lambda x: str(x)[:10]if x is not None else None)

In [42]:
df_listorder['is_late']= df_listorder.apply(lambda x: isLate(x['day_pick'], x['day_1st_deliver_attempt'],x['day_2nd_deliver_attempt'],x['SLA']), axis=1)

In [44]:
df_listorder[['orderid','is_late']].to_csv('./data/resultFinal.csv',index=False)